In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

First the usual boiler plate code.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

let's import necessary modules

In [ ]:
from pathlib import Path
from fastai.vision import *
from fastai.metrics import error_rate

Let's look into the dataset.

In [ ]:
def dataPath(path, pathToDataset):
     path = Path(path)
     datasetPath = path/pathToDataset
     return path, datasetPath, datasetPath.ls()

In [ ]:
path = '../input'
path, datasetPath, subdirectories = dataPath(path, 'waffles-or-icecream')
(path, datasetPath, subdirectories)

The below function returns a list of filenames which are of valid image file extensions

In [ ]:
def listifyFileNames(subdirectoryPath):
    
    fileNames = [fileName for i in range(len(subdirectoryPath)) for fileName in subdirectoryPath[i].ls()]
    validExtensions = ['.jpg', '.jpeg', '.png', '.JPG', '.jpeg', '.PNG']
    validFileNames = list(filter(lambda fileName: fileName.suffix in validExtensions, fileNames))
    
    return validFileNames

The below function returns a list of lables for each image file in the list from the previous step.

In [ ]:
def listifyLabels(fileNamesList):
    return ['ice-cream' if '/ice_cream/' in str(fileName) else 'waffles' for fileName in fileNamesList]

It's time to look inisde those directories and find out what we got.

In [ ]:
def createDataBunch(path, filePathList, labelFunc, percentOfDataToSplit, imageSize):
     return ImageDataBunch.from_lists(path,
                                        filePathList,
                                        labels = labelFunc,
                                        ds_tfms=get_transforms(),
                                        valid_pct=percentOfDataToSplit,
                                        size=imageSize).normalize(imagenet_stats)

In [ ]:
fileNamesList = listifyFileNames(subdirectories)
data = createDataBunch(datasetPath, fileNamesList, listifyLabels(fileNamesList),0.2, 224)
data.classes

In [ ]:
print('ice-cream- {}'.format(len(subdirectories[0].ls())))
print('waffles- {}'.format(len(subdirectories[1].ls())))

In [ ]:
#TODO: need to modify the above fucntion as it is returning only icecreams
data.show_batch(rows=3, figsize=(7,6))

* #Transfer learning

Using the pre-trained model created for waste classification to train the network to classify the ice-cream vs waffles.
My thought behind this is that -->
 * The pre-trained model was traine dto do binary classification. So, it contains the knowledge about differentiating between two different classes which could help me in this scenario.

1. create a CNN learner object with a base architecture.

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
preTrainedModelPath = 'pretrained-model-for-classifying-types-of-trash'
preTrainedModelPath

set the path to the pre-trained model

In [ ]:
learn.path = path
learn.model_dir = preTrainedModelPath
learn.model_dir

load the pre-trained model

In [ ]:
learn.load('final')

train for 4 epochs

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
image = learn.data.valid_ds[3][0]
image

predict on a atest image

In [ ]:
learn.predict(image)

this function saves the trained model to a directory.

In [ ]:
def saveModel(learnerObject, model_dir, modelName= None, export: bool= True,return_path: bool= True):
    learnerObject.model_dir = Path(model_dir)
    
    if export is True:
        learnerObject.path = Path(model_dir)
        learnerObject.export()
    else:
        learnerObject.save(modelName, return_path=return_path)

In [ ]:
saveModel(learn, "/kaggle/working", 'stage-1', False, False)

this function plots the images where the classification went wrong and the corresponding confusion matrix.

In [ ]:
def plotTopLossesAndConfusionMatrix(learnerObject, data, numOfRows, figureSize: tuple, confMatrixSize: tuple, dpi, plotConfusionMatrix: bool = True):
    interp = ClassificationInterpretation.from_learner(learnerObject)
    losses,idxs = interp.top_losses()
    len(data.valid_ds)==len(losses)==len(idxs)
    interp.plot_top_losses(numOfRows, figsize=figureSize)
    
    if plotConfusionMatrix:
        interp.plot_confusion_matrix(figsize=confMatrixSize, dpi=dpi)
        
    interp.most_confused(min_val=2)

In [ ]:
plotTopLossesAndConfusionMatrix(learn, data, 9, (15,11), (11,11), 60)

training the model for one epoch and then plotting the current learning rate.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1)

In [ ]:
learn.load('stage-1')
learn.lr_find()
learn.recorder.plot()

unfreezing the model to train the backbone of the pre-trained model with a range of learning rate deduced from the above plot.

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-5,1e-3))

In [ ]:
plotTopLossesAndConfusionMatrix(learn, data, 9, (15,11), (11,11), 60)

The performance looks good now.

export the model for inference.

In [ ]:
saveModel(learn, "/kaggle/working", False)